In [1]:
!pip install ultralytics ncnn
import os
import random
import yaml
import shutil
from google.colab import drive
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 60.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
# 2. CONFIGURAÇÃO (pessoal de visão so deve mexer aqui)
NOME_ZIP = "plataforma_de_voo_v1.zip" # @param {type:"string"}

# ---
PATH_DRIVE = f"/content/{NOME_ZIP}"
PATH_LOCAL = "/content/dataset_raw"
PATH_FINAL = "/content/yolo_dataset"

In [6]:
def prepare_dataset(zip_path, local_path, final_path, split_ratio=0.2):
    # 1. Limpeza de ambiente
    if os.path.exists(local_path): shutil.rmtree(local_path)
    if os.path.exists(final_path): shutil.rmtree(final_path)
    os.makedirs(local_path, exist_ok=True)

    # 2. Unzip com tratamento de erro de caminho
    print(f"📦 Extraindo {zip_path}...")
    if not os.path.exists(zip_path):
        raise FileNotFoundError(f"❌ O arquivo ZIP não foi encontrado no Drive: {zip_path}")

    !unzip -q "{zip_path}" -d {local_path}

    # 3. Busca exaustiva pela pasta de dados (Ignora Case e caminhos aninhados)
    src_path = None
    for root, dirs, files in os.walk(local_path):
        for d in dirs:
            if d.lower() == "obj_train_data": # Força busca em lowercase
                src_path = os.path.join(root, d)
                break
        if src_path: break

    if not src_path:
        # Debug: Listar o que foi extraído para te ajudar a ver o erro
        print("Conteúdo detectado no ZIP:")
        !ls -R {local_path} | head -n 20
        raise FileNotFoundError("❌ A pasta 'obj_train_data' não foi encontrada! Verifique o log acima.")

    # 4. Criar estrutura padrão YOLOv11
    for folder in ['images/train', 'images/val', 'labels/train', 'labels/val']:
        os.makedirs(os.path.join(final_path, folder), exist_ok=True)

    # 5. Processamento de Imagens e Labels
    all_files = os.listdir(src_path)
    images = [f for f in all_files if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    random.shuffle(images)

    split_idx = int(len(images) * (1 - split_ratio))
    train_images = images[:split_idx]
    val_images = images[split_idx:]

    def move_pair(file_list, subset):
        count = 0
        for img_name in file_list:
            img_src = os.path.join(src_path, img_name)
            # Tenta achar o .txt correspondente (mesmo nome da imagem)
            label_name = os.path.splitext(img_name)[0] + ".txt"
            label_src = os.path.join(src_path, label_name)

            if os.path.exists(label_src):
                shutil.move(img_src, os.path.join(final_path, f"images/{subset}/{img_name}"))
                shutil.move(label_src, os.path.join(final_path, f"labels/{subset}/{label_name}"))
                count += 1
        return count

    t_count = move_pair(train_images, 'train')
    v_count = move_pair(val_images, 'val')

    print(f"\n✅ Reestruturação concluída com sucesso!")
    print(f"📊 Treino: {t_count} | Validação: {v_count}")

prepare_dataset(PATH_DRIVE, PATH_LOCAL, PATH_FINAL)

📦 Extraindo /content/plataforma_de_voo_v1.zip...

✅ Reestruturação concluída com sucesso!
📊 Treino: 1271 | Validação: 318


In [ ]:
import yaml
import os

# 1. Limpeza de caches antigos para evitar o RuntimeError
!find {PATH_FINAL} -name "*.cache" -delete

# 2. Criar dataset.yaml (Garantindo que os caminhos são relativos ao 'path')
config = {
    'path': PATH_FINAL,
    'train': 'images/train',
    'val': 'images/val',
    'names': {
        0: 'plataforma-voo'
    }
}

with open('/content/dataset.yaml', 'w') as f:
    yaml.dump(config, f)

# 3. Iniciar Treino com YOLOv11 Nano
model = YOLO("yolo11n.pt")

# Verificação de segurança: printar o caminho para conferência
print(f"🚀 Iniciando treino usando o dataset em: {PATH_FINAL}")

model.train(
    data="/content/dataset.yaml",
    epochs=100,
    imgsz=640,
    device=0, # Certifique-se que o T4 GPU está ativo
    project=f"/content/drive/MyDrive/0-Eletroquad2026/VCOMP/experimentos",
    name=f"run_plataforma_voo",
    exist_ok=True # Permite sobrescrever se a pasta já existir
)

# 4. Exportar para NCNN
model.export(format="ncnn", imgsz=640)
print("🚀 Modelo exportado com sucesso para o Drive!")

🚀 Iniciando treino usando o dataset em: /content/yolo_dataset
Ultralytics 8.3.243 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=run_plataforma_voo, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, ove

In [ ]:
# Verificação de arquivos
print("--- Imagens no Treino ---")
!ls /content/yolo_dataset/images/train | head -n 5
print("\n--- Labels no Treino ---")
!ls /content/yolo_dataset/labels/train | head -n 5
print("\n--- Conteúdo de uma label (exemplo) ---")
!cat /content/yolo_dataset/labels/train/*.txt | head -n 1